<a href="https://colab.research.google.com/github/HaoYamado/notebooks/blob/master/detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
precision = 'fp32'
ssd_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd', model_math=precision)

In [0]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_ssd_processing_utils')

In [0]:
ssd_model.to('cuda')
ssd_model.eval()

In [0]:
uris = [
    'https://sun9-63.userapi.com/c855032/v855032749/15737c/P3EqZfej4yk.jpg',
    'https://sun9-3.userapi.com/c635107/v635107068/1da79/AOYtD6wX-YA.jpg',
]

In [0]:
inputs = [utils.prepare_input(uri) for uri in uris]
tensor = utils.prepare_tensor(inputs, precision == 'fp16')

In [0]:
with torch.no_grad():
    detections_batch = ssd_model(tensor)

In [0]:
results_per_input = utils.decode_results(detections_batch)
best_results_per_input = [utils.pick_best(results, 0.40) for results in results_per_input]

In [0]:
classes_to_labels = utils.get_coco_object_dictionary()

In [0]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os
path = '/home/haoyamado'
for image_idx in range(len(best_results_per_input)):
    fig, ax = plt.subplots(1)
    # Show original, denormalized images...
    image = inputs[image_idx] / 2 + 0.5
    ax.imshow(image)
    # ..with detections
    bboxes, classes, confidences = best_results_per_input[image_idx]
    for idx in range(len(bboxes)):
        left, bot, right, top = bboxes[idx]
        x, y, w, h = [val * 300 for val in [left, bot, right - left, top - bot]]
        rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x, y, '{} {:.0f}%'.format(classes_to_labels[classes[idx] - 1], confidences[idx]*100), bbox=dict(facecolor='white', alpha=0.5))
plt.show()
plt.savefig('11101.png')
plt.savefig('11102.png')
plt.savefig('11103.png')